#Csomagok letöltése

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

#Tanítóhalmaz beszerzése és tokenizálása
Az mrpc adathalmazt használjuk, amely a GLUE benchmark egyik adathalmaza.\
Betöltjük a checkpointokat.\
Tokenizáljuk a mondatokat. Jelenleg két mondatunk is van egy sorban, így mindkettőt betöltjük a tokenizernek, ami kéeps kezelni mindkettőt. \
A tokenized_datasets-el létrejön egy bővített dictionary, amiben már van input_ids, token_type_ids, attention_mask. \
A data_collator pedig képes arra, hogy a létrehozott batchekben paddingeljen, tehát megtalálja a legtöbb tokenszámmal rendelkező mondatot.

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#Az adathalmaz formázása
A cél, hogy olyan formátumban legyen az adathalmaz, amit a PyTorchos Trainer elvár. \
Eltávolítjuk a nem szükséges oszlopokat.\
Átnevezzük a label oszlopot labels-re. Ez azért kell, mert a Trainer is ilyen nevű oszlopot keres majd. \
Át kell alakítanunk az adathalmaz formátumát torch-á, hogy a numerikus mezők torch.Tenzor typusúak legyenek. Mivel a Trainer és a modell a PyTorch-os tanításhoz tenzorokat vár, nem listákat.\
A végén ellenőrizzük, hogy csak a szükséges oszlopok maradtak-e.




In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

#Az adathalmaz átalakítása DataLoaderekké
A DataLoader a Pytorch egyik fontos eszköze, ami batcheket készít, és szolgáltatja az adatot a modellnek a training közben.
1. Létrehozza a batcheket a megadott mérettel
2. Ha a shuffle=True, akkor minden epoch elején összekeveri a mintákat, hogy más sorrendben lássa a modell.
3. A collate_fn=data_collator teszi lehetővé a paddingot \
Ezzel a lépéssel tehát előkészítjük a tanító és validációs halmazokat batchekre osztva.

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

#A DataLoader és a DataCollator működésének ellenőrzése
Itt minden egyes batch-re megnézzük, hogy jól működik-e a paddingelés. \
Megmutatja, hogy az input_ids, attention_mask és labels
mind helyes méretű PyTorch tensorok-e.

In [ ]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

#A modell betöltése

Az AutoModelForSequenceClassification egy olyan modellosztály,
ami egy BERT-szerű alapmodellt (pl. DistilBERT, RoBERTa, DeBERTa stb.)
kiegészít egy klasszifikációs fejjel a tetején. \
A num_labels-el megmondjuk, hogy ez egy kétosztályos (bináris) osztályozási feladat: pozitív / negatív. \
Betöltjük neki a checkpointokat is, hogy ott folytassa, ahol abbahagyta a tanulást.

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

#Egy batch kipróbálása
A **batch Python-szintaxis annyit csinál, hogy kinyitja a dictionary-t kulcs–érték párokra. Tehát a dictionary minden kulcsa a megfelelő argumentumba kerül. a modellnél. \
Az outputs egy olyan objektum, ami az adott batchhez kiszámolja a losst, és a logits-okat. \
A loss kiszámolja az adott batch-hez tartozó veszteséget. \
A logits érték pedig egy nyers érték, ami mond valamit a labelekről. Ez még nem valószínűség, azt majd a softmax számolja ki a logitsokból. \
Az outputs.loss megmutatja a veszteség értékét. \
Az outputs.logits.shape megmutatja a modell kimenetelének méretét, ami így néz ki: [batch, osztályok]. Mivel a batch méret 8, az osztályokból pedig 2 van, ennek így kell kinéznie: [8, 2]

In [ ]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

#Az optimalizáló létrehozása
Az optimalizáló feladata, hogy frissítse a modell súlyait minden bacth után. \
Az optimalizáló algoritmus neve AdamW \
Megadjuk neki a model.parameters()-t, ami átadja az optimalizálónak az összes tanítható súlyt. \
A lr=5e-5 a tanulási ráta (learning rate). \
Az optimalizáló a loss.backward() függvény meghívásával végzi el a súlyfrissítést.

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

#A tanulási ráta beállítása
Ez a „tanulási ráta ütemező” (learning rate scheduler) teszi lehetővé, hogy a modell fokozatosan, stabilan tanuljon, ne ugorjon össze-vissza. \
Az optimizernek volt egy paramétere, az lr=5e-5, ami megmondta, mekkora legyen a tanulási ráta. Viszont szeretnénk, ha ez nem lenne végig ugyanaz, hanem fokozatosan csökkenjen a tanítás során. Erre való az LR scheduler. \
num_epochs=3 megmondja, hogy a teljes adathalmazon 3-szor menjünk végig. \
A num_training_steps kiszámolja, hogy hány lépés lesz összesen. A len(train_dataloader) kiszámolja, hogy hány batch van összesen, és ezt megszorozza az epochok számával. Tehát ennyi lépés lesz és ennyiszer lesz súlyfrissítés is. \
A "linear" típus azt jelenti, hogy a tanulási ráta lineárisan csökken a kezdeti értéktől a 0-ig a tanulás során. \
A num_warmup_steps=0 megadja, hogy nincs fokozatos bemelegítés a tanulási rátának. \
Tehát a cél a scheduler-rel, hogy fokozatosan csökkentsük a tanulási rátát az epochok során, ezzel elérjük a stabilabb, hatékonyabb tanulást.


In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

#Melyik eszközön fusson a tanítás?
Betöltjük a PyTorch könyvtárat, ami kezeli a számítási műveleteket. (Tenzorok, GPU, stb.) \
Ha..akkor kifejezéssel megvizsgáljuk, hogy van-e elérhető GPU a rendszerben. Ha van, akkor a GPU-t fogja használni (pl. NVIDIA Tesla T4 a Colabban), ha nincs, akkor minden számítás a processzoron (CPU) fog futni. \
A model.to(device) átmozgatja a modellt a megfelelő eszközre. \
A device kiírja, melyiket fogja használni.

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

#A tényleges tanítás
Több epochon keresztül minden batch-en végighaladva kiszámoljuk a veszteséget, visszaterjesztjük a hibát, frissítjük a súlyokat és csökkentjük a tanulási rátát. \
A tqdm egy folyamatjelző könyvtár, ami szép, automatikusan frissülő csíkot ad, ami mutatja, hány lépés van még hátra. \
Beadjuk neki a num_training_steps hosszát, ami azt mondja meg, hogy még hány batch lesz összesen. Tehát a csík minden batch után 1-et lép előre. \
A model.train()-el átváltjuk a modellt tréning módba. (Amikor értékelünk, akkor majd a model.eval()-ra lesz szükség. \
Két ciklus definiálunk: minden epochra fusson le a teljes tanítás, és az eopchokon belül minden batchre végezzük el a szükséges lépéseket. \
A batch = {k: v.to(device) for k, v in batch.items()} áthelyezi az egész batchet a GPU-ra. \
A outputs = model(**batch) egy adott batchre lefuttatja a modellt, így lesz egy loss értéke és kiszámítja az előrejelzéseket (logitsokat). \
A loss=outputs.loss -al kivesszük a loss értékeket a kimenetből. \
A loss.backward() kiszámolja, hogy milyen irányba kell módosítani a súlyokat, hogy a loss csökenjen. \
Az optimizer.step() ténylegesen elvégzi a súlyfrissítést. \
A lr.scheduler.step() ténylegesen frissíti a tanulási rátát. \
Az optimizer.zero_grad() kinullázza minden batch után a gradiens értékét. Ezt azért kell elvégezni, mert a Pytorch nem írja felül, hanem összeadja a gradiens értékeket, tehát felhalmozódna a gradiens értéke. \
A progress_bar.update(1) pedig minden batch után 1 lépést halad előre.





In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

#A modell értékelése
A cél: lefuttatni a modellt a validációs adathalmazon, jóslatokat készíteni, és lemérni, hogy mennyire jók a predikciók. \
Betöltjük az evaluate könyvtárat. \
A metric = evaluate.load("glue", "mrpc") letölti és előkészíti a GLUE banchmark egyik metrikáját, ami az mrcp feladathoz tartozik. A feladat azt méri, hogy a két mondat megegyezik-e vagy sem. A metrika több értékelést is visszaad, pl. pontosság, F1 score, stb. \
A model.eval() átállítja a modellt kiértékelő módra. \
Az eval_dataloader az evaluation adathalmaz előkészített változatát adja, ezen futtatjuk batchenként a kiértékelést. \
A batch = {k: v.to(device) for k, v in batch.items()} átteszi az összes batch elemet az adott eszközre. \
 A with torch.no_grad() függvénnyel megmondjuk, hogy ne tanuljon (tehát ne számoljon gradienseket), hanem csak előrejelzést számoljon. Ez gyorsabb és kevesebb memóriát használ. \
 logits=outputs.logits -al. \
 A predictions = torch.argmax(logits, dim=-1) kiszedi a legnagyobb logitot minden mintánál. Ez adja meg az előrejelzést (mert ahol a legnagyobb a logit, az a legvalószínűbb érték). \
 A metric.add_batch(predictions=predictions, references=batch["labels"]) sorral eltároljuk az eredményt minden batch esetén (az előrejelzést és az igazi címkét). A végén az összes batch eredményével együtt tudja kiértékelni a modellt az evaluate könyvtár. \
 A metric.compute() hívásával kiszámoljuk az összesített eredményt.

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

#A teljes tanítási ciklus


In [ ]:
from torch.optim import AdamW
from transformers import AutoModelForSequenceClassification, get_scheduler

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)